# Summarising geometry

With a data cube containing shapely geoemtry objects as data, you can use `xvec` to generate a summary geometry that can be used as an additional `GeometryIndex`.

In [ ]:
import xarray as xr
import numpy as np
import shapely
import xproj
import xvec

Generate a toy cube.

In [8]:
cube = xr.DataArray(
    shapely.points(np.tile(np.arange(0, 20, 4), 5), np.repeat(np.arange(5), 5)).reshape(5, 5),
    coords=[["a", "b", "c", "d", "e"], ["2020-10-01", "2020-10-02", "2020-10-03", "2020-10-04", "2020-10-05"]],
    dims=["gid", "date"])
cube

<xarray.DataArray (gid: 5, date: 5)> Size: 200B
array([[<POINT (0 0)>, <POINT (4 0)>, <POINT (8 0)>, <POINT (12 0)>,
        <POINT (16 0)>],
       [<POINT (0 1)>, <POINT (4 1)>, <POINT (8 1)>, <POINT (12 1)>,
        <POINT (16 1)>],
       [<POINT (0 2)>, <POINT (4 2)>, <POINT (8 2)>, <POINT (12 2)>,
        <POINT (16 2)>],
       [<POINT (0 3)>, <POINT (4 3)>, <POINT (8 3)>, <POINT (12 3)>,
        <POINT (16 3)>],
       [<POINT (0 4)>, <POINT (4 4)>, <POINT (8 4)>, <POINT (12 4)>,
        <POINT (16 4)>]], dtype=object)
Coordinates:
  * gid      (gid) <U1 20B 'a' 'b' 'c' 'd' 'e'
  * date     (date) <U10 200B '2020-10-01' '2020-10-02' ... '2020-10-05'

Assign CRS to the array using `xproj`.

In [9]:
cube = cube.proj.assign_crs(spatial_ref=3857)
cube

<xarray.DataArray (gid: 5, date: 5)> Size: 200B
array([[<POINT (0 0)>, <POINT (4 0)>, <POINT (8 0)>, <POINT (12 0)>,
        <POINT (16 0)>],
       [<POINT (0 1)>, <POINT (4 1)>, <POINT (8 1)>, <POINT (12 1)>,
        <POINT (16 1)>],
       [<POINT (0 2)>, <POINT (4 2)>, <POINT (8 2)>, <POINT (12 2)>,
        <POINT (16 2)>],
       [<POINT (0 3)>, <POINT (4 3)>, <POINT (8 3)>, <POINT (12 3)>,
        <POINT (16 3)>],
       [<POINT (0 4)>, <POINT (4 4)>, <POINT (8 4)>, <POINT (12 4)>,
        <POINT (16 4)>]], dtype=object)
Coordinates:
  * gid          (gid) <U1 20B 'a' 'b' 'c' 'd' 'e'
  * date         (date) <U10 200B '2020-10-01' '2020-10-02' ... '2020-10-05'
  * spatial_ref  int64 8B 0
Indexes:
    spatial_ref  CRSIndex (crs=EPSG:3857)

Generate summary geometry along 'gid' dimension.

In [10]:
cube = cube.xvec.summarize_geometry('gid')
cube

<xarray.DataArray (gid: 5, date: 5)> Size: 200B
array([[<POINT (0 0)>, <POINT (4 0)>, <POINT (8 0)>, <POINT (12 0)>,
        <POINT (16 0)>],
       [<POINT (0 1)>, <POINT (4 1)>, <POINT (8 1)>, <POINT (12 1)>,
        <POINT (16 1)>],
       [<POINT (0 2)>, <POINT (4 2)>, <POINT (8 2)>, <POINT (12 2)>,
        <POINT (16 2)>],
       [<POINT (0 3)>, <POINT (4 3)>, <POINT (8 3)>, <POINT (12 3)>,
        <POINT (16 3)>],
       [<POINT (0 4)>, <POINT (4 4)>, <POINT (8 4)>, <POINT (12 4)>,
        <POINT (16 4)>]], dtype=object)
Coordinates:
  * gid               (gid) <U1 20B 'a' 'b' 'c' 'd' 'e'
  * date              (date) <U10 200B '2020-10-01' ... '2020-10-05'
  * spatial_ref       int64 8B 0
  * summary_geometry  (gid) object 40B POLYGON ((0 0, 16 0, 16 0, 0 0, 0 0)) ...
Indexes:
    spatial_ref       CRSIndex (crs=EPSG:3857)
    summary_geometry  GeometryIndex (crs=EPSG:3857)

The default summary is using the total envelope (bounding box) of all geometries at the same coordinate of the set dimension. However, you can do any of the:

- `"envelope"` - envelope (bounding box) of combined geometries
- `"oriented_envelope"` - oriented envelope (minimum rotated recatngle) of combined geometries
- `"centroid"`- centroid of combined geometries (derived from a GeometryCollection combining all)
- `"convex_hull"` - convex hull of combined geometries
- `"collection"` - collection combining all geometries
- `"union"` - union of all geometries

In [11]:
cube.xvec.summarize_geometry('gid', 'centroid')

<xarray.DataArray (gid: 5, date: 5)> Size: 200B
array([[<POINT (0 0)>, <POINT (4 0)>, <POINT (8 0)>, <POINT (12 0)>,
        <POINT (16 0)>],
       [<POINT (0 1)>, <POINT (4 1)>, <POINT (8 1)>, <POINT (12 1)>,
        <POINT (16 1)>],
       [<POINT (0 2)>, <POINT (4 2)>, <POINT (8 2)>, <POINT (12 2)>,
        <POINT (16 2)>],
       [<POINT (0 3)>, <POINT (4 3)>, <POINT (8 3)>, <POINT (12 3)>,
        <POINT (16 3)>],
       [<POINT (0 4)>, <POINT (4 4)>, <POINT (8 4)>, <POINT (12 4)>,
        <POINT (16 4)>]], dtype=object)
Coordinates:
  * gid               (gid) <U1 20B 'a' 'b' 'c' 'd' 'e'
  * date              (date) <U10 200B '2020-10-01' ... '2020-10-05'
  * spatial_ref       int64 8B 0
  * summary_geometry  (gid) object 40B POINT (8 0) POINT (8 1) ... POINT (8 4)
Indexes:
    spatial_ref       CRSIndex (crs=EPSG:3857)
    summary_geometry  GeometryIndex (crs=EPSG:3857)